<a href="https://colab.research.google.com/github/mugalan/vibration-analysis/blob/main/assignments/Basic_Vibration_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>